In [ ]:
"""
It seems that Plotly makes possible to define its objects, both using the Class 
formulation or the Dictionary formulation.

For example, to plot with Plotly, I can pass a Figure object to the function, or 
I can pass a dictionary containing the components of the Figure object, which 
are Data and Layout.

The Figure class contains Data and Layout. The Data can be expressed as a 
Trace or as a Dictionary. 

A "Trace" is an Plotly object such as Scatter(), Bar(), etc...

                            +--> Trace
                +--> Data --+
                |           +--> [{type: '', }. {type: ''}]
                |
Plot --> Figure +
                |
                |             +--> Layout
                +--> Layout --+ 
                              +--> Dict()
    
    
    
"""

In [1]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, './')

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import init_notebook_mode, iplot

# Import stuff for making the plot interactive
from ipywidgets import interactive, HBox, VBox, widgets, Label, Layout
import numpy as np

init_notebook_mode(connected=True)

In [2]:
from trjgen import trjgen_helpers as tjh
from trjgen import pltly_helpers as ply
from trjgen import pwpoly as pw

In [3]:
# Polynomial characteristic:  order
ndeg = 7
print('Test with {:d}th order polynomial.'.format(ndeg))

## Waipoints in the flat output space (or dimension 3)
nconstr = 4
print('Number of constraint on the flat output = {:d}.'.format(nconstr))


X = np.array([
        [ 0,     1.0,    0.5],
        [ 0,     0.8,   0.0],
        [ 0,   -5.0,     0.0],
        [ 0,   np.nan,   0.0],
        ])

Y = np.array([
        [ 0,   0.0,      0.0],
        [ 0,   0.0,      0.0],
        [ 0,   0.0,      0.0],
        [ 0,   np.nan,   0.0],
        ])

Z = np.array([
        [ 0,   0.5,     -0.2],
        [ 0,   0.0,      0.0],
        [ 0,   0.0,      0.0],
        [ 0,   np.nan,   0.0],
        ])

W = np.array([
        [ 0,   np.nan,      0.0],
        [ 0,   np.nan,      0.0],
        [ 0,   np.nan,      0.0],
        [ 0,   np.nan,      0.0],
        ])


Test with 7th order polynomial.
Number of constraint on the flat output = 4.


In [9]:
## Evaluate initial data
vehicle_mass = 0.032
thrust_thr = 9.91 * 0.032 * 2.0

mid = 1.5
Tmax = 3
# Times (Absolute and intervals)
knots = np.array([0, mid, Tmax]) # One second each piece

# Check (Evaluate polynomial)
tv = np.linspace(0,max(knots),100);

ppx = pw.PwPoly(X, knots, ndeg)
ppy = pw.PwPoly(Y, knots, ndeg)
ppz = pw.PwPoly(Z, knots, ndeg)
ppw = pw.PwPoly(W, knots, ndeg)
    
# Generate the first trajectory
(Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)

# Compute the thrust limits
(ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)

perc_available = (available_thrust / thrust_thr) * 100.0

In [13]:
# Define initial TRACES and DATA
#
# Figure {
#   Data
#   Layout
#}
#
# Data {
#   Trace
#   Type
#}
#
# The Trace describes the kind of representation and the data to be represented.
#

# Defined as an object
trace1 = go.Scatter3d(
            x = Xtj[0, :],
            y = Ytj[0, :],
            z = Ztj[0, :],
            mode = 'markers',
            marker = dict(
                size = 4,
                color = tv,
                colorscale = 'Viridis',
                opacity = 0.8,
                colorbar = dict (
                    thickness = 20,
                    len =  0.5,
                    x = 0.8,
                    y = 0.6
                    )
                ),
            name = '3D Path'
            )

# Defined as an object
trace2 = {
            "type": "cone",
            "x": Xtj[0,:],
            "y": Ytj[0,:],
            "z": Ztj[0,:],
            "u": Zbtj[0,:],
            "v": Zbtj[1,:],
            "w": Zbtj[2,:],
            "sizemode": "absolute",
            "colorscale": 'Blues',
            "sizeref": 10000,
            "visible": False
    }

trace3 = go.Scatter3d(
        x = Xtj[0, :],
        y = Ytj[0, :],
        z = Ztj[0, :],
        mode = 'markers',
        marker = dict(
            size = 4,
            color = perc_available,
            colorscale = 'Viridis',
            opacity = 0.8,
            colorbar = dict ( thickness = 20, len =  0.5, 
                             x = 0.8, y = 0.6)
            ),
        name = '3D Path with thrust free margin',
        visible = False
        )

# Data is a list of 
data = [trace1, trace2, trace3]

In [41]:
## Interactive element and relative methods
#
# 1) Slider to select the intercept time
# 2) Box to select the max time
# 

# 1) Plot selector Widget
plotSelector_w = widgets.Dropdown(
    options=['Time', 'Attitude', 'Thrust Limit'],
    value='Time',
    description='Plot Type',
)

# 2) Box for final time Widget
boxFTime_w = widgets.FloatText(
    value = 3.0,
    description = "Final Time"
)

# 3) Slider for intercept time Widget
sliderITime_w = widgets.FloatSlider(min=0.0, max=3.0, step=0.1, value=1.5, description = "Intercept Time", width=200)

#slider_w = interactive(update_data, intercept_time = (0.0, Tmax, 0.1), description = "Intercept Time")




## CALLBACKS

# This callback modify the final time of the trajectory
def finalTime_response(change):
    sliderITime_w.max = boxFTime_w.value


def plotSelector_response(change):
#    with figure_w.batch_update():
    if (plotSelector_w.value == 'Time'):
        figure_w.data[0].visible = True
        figure_w.data[1].visible = False
        figure_w.data[2].visible = False
    if (plotSelector_w.value == 'Attitude'):
        figure_w.data[0].visible = False
        figure_w.data[1].visible = True
        figure_w.data[2].visible = False
    if (plotSelector_w.value == 'Thrust Limit'):
        figure_w.data[0].visible = False
        figure_w.data[1].visible = False
        figure_w.data[2].visible = True



def update_data(change):
    mid = sliderITime_w.value 
    knots = np.array([0, mid, 3]) # One second each piece
    ppx = pw.PwPoly(X, knots, ndeg)
    ppy = pw.PwPoly(Y, knots, ndeg)
    ppz = pw.PwPoly(Z, knots, ndeg)
    ppw = pw.PwPoly(W, knots, ndeg)
    
    (Xtj, Ytj, Ztj, Wtj, Zbtj) = tjh.TrajFromPW(tv, [0,1,2], \
        pwpolx=ppx, pwpoly=ppy, pwpolz=ppz, pwpolw = ppw)
    
    (ffthrust, available_thrust) = tjh.getlimits(tv, Xtj, Ytj, Ztj, \
        vehicle_mass, thrust_thr)
    perc_available = (available_thrust / thrust_thr) * 100.0

    figure_w.data[0].x = Xtj[0, :]
    figure_w.data[0].y = Ytj[0, :]
    figure_w.data[0].z = Ztj[0, :]
    figure_w.data[0].marker.color = tv
    
    figure_w.data[1].x = Xtj[0, :]
    figure_w.data[1].y = Ytj[0, :]
    figure_w.data[1].z = Ztj[0, :]
    figure_w.data[1].u = Zbtj[0, :]
    figure_w.data[1].v = Zbtj[1, :]
    figure_w.data[1].w = Zbtj[2, :]
        
    figure_w.data[2].x = Xtj[0, :]
    figure_w.data[2].y = Ytj[0, :]
    figure_w.data[2].z = Ztj[0, :]
    figure_w.data[2].marker.color = perc_available





# Now I have to link the widgets to callbacks
boxFTime_w.observe(finalTime_response, names = 'value')
plotSelector_w.observe(plotSelector_response, names="value")
sliderITime_w.observe(update_data, names='value')



In [44]:
## Define Plot LAYOUT

mylayout = go.Layout(
    autosize=True,
    width=800,
    height=800,
    margin = dict( l = 0, r = 0, b = 0, t = 0),
    scene={"aspectmode": "data",
                "xaxis": {"title": f"x [m]"},
                "yaxis": {"title": f"y [m]"},
                "zaxis": {"title": f"z [m]"}
           },
    xaxis=go.layout.XAxis(
        automargin=True
    ),
    yaxis=go.layout.YAxis(
        automargin=True
    )
)

figure = go.Figure(data=data, layout=mylayout)

# Interactive elements
label_layout = Layout(width='600px',height='50px')
menu1_container = widgets.HBox([plotSelector_w, boxFTime_w], layout = label_layout)
menu2_container = widgets.HBox([sliderITime_w])
figure_w = go.FigureWidget(figure)

    
# display the FigureWidget and slider with center justification
vb = VBox((menu1_container, menu2_container, figure_w))
vb.layout.align_items = 'center'
vb